In [43]:
from itertools import product
import netCDF4 as ncf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import math
import csv
from tqdm import tqdm_notebook as tqdm
from keras_tqdm import TQDMNotebookCallback
import os
import random
import tempfile
import pandas as pd
from joblib import dump, load

In [44]:
a=[2,4,7,9,13,17,19]        #a is the hour for which we're trying to predict
b=[0,1,2,3,4,5,6]      # b is indexing for a

#### dates generator. 6 weeks.
no_files=5
nfiles_train=[]
nfiles_test=[]
day_count=5
no_of_months=1
for i in range(no_of_months):
    nfiles_temp=list(range(day_count,day_count+15))  #7
    nfiles_train.append(nfiles_temp)
    nfiles_temp=list(range(day_count+16,day_count+22))  #11
    nfiles_test.append(nfiles_temp)
    day_count=day_count+30
# find days of the year

In [45]:
##  Grid reference both x-axis,y-axis
loc_y = [165,121,171]
loc_x = [67,58,39]

grid_size_x = dx = 20
grid_size_y = dy = 20

grid_sample_limit_x=[]
grid_sample_limit_y=[]
for i in loc_x:
    grid_sample_limit_x.append((i-15,i+15))
for i in loc_y:
    grid_sample_limit_y.append((i-15,i+15))

grids_x = [np.arange(x-dx, x+dx, 1) for x in loc_x]
grids_y = [np.arange(y-dy, y+dy, 1) for y in loc_y]

all_grids_x = np.concatenate(grids_x)
all_grids_y = np.concatenate(grids_y)

In [46]:
monitors_file = 'C:/Users/woshi/Desktop/CE_675_Project/Data/loc_data/monitor_list_NC.csv'
with open(monitors_file) as csvfile:
    reader = csv.DictReader(csvfile)
    monitor_points = [(int(float(row['Col'])), int(float(row['Row'] ))) for row in reader if row['Col']] 

monitors_x, monitors_y = zip(*monitor_points)

all_grids_x=np.concatenate([all_grids_x,monitors_x])
all_grids_y=np.concatenate([all_grids_y,monitors_y])

all_points = list(product(all_grids_x, all_grids_y))

In [47]:
#function to get the entire file list in a directory
def get_file_list(data_dir):
    return [os.path.join(data_dir, name) for name in os.listdir(data_dir)]
#function to get index for a point based on grid reference.
def get_index(grid,val):
    try:
        return np.where(grid==val)[0][0]
    except (TypeError,IndexError):
            if(len(val)>1):
                index_list=[]
                for i in val:
                    index_list.append(np.where(grid==i)[0][0])
                return index_list

In [48]:
short_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/short_data/'

short_files = get_file_list(short_dir)
#short_files=short_files[:30]
#it's the list of list of all indices for the hours to be predicted everyday

In [49]:
# generate 10 time points

def list_seg(n,look_back,lb_ndarr):
    t_part_list=[]
    i=0
    tlist=lb_ndarr[n]
    def creating_tlist(tlist,j,ind):
        try:
            ind=tlist.index(0-j*24)
            t_part_list.append([i+(0+j*24) for i in tlist[ind:]])
            tlist=tlist[:ind]
            j=j+1
            return creating_tlist(tlist,j,ind)
        except ValueError:
            t_part_list.append([i+(0+j*24) for i in tlist[:ind]])
            return t_part_list
    t_part_list=creating_tlist(tlist,i,0)
    return t_part_list #returns the required sets of indices for a particular hour at request

def hours_list(a,l_b,f_h):
    lb_ndarr = []
    for i in a:
        tlist=list(range(i-l_b,i))           # this is where first_ndarr is created
        lb_ndarr.append(tlist)
    fh_ndarr=[] #this is the list of all indices for the future data collection
    for i in a:
        tlist=list(range(i+1,i+f_h+1))
        fh_ndarr.append(tlist)
    return lb_ndarr,fh_ndarr

In [50]:
def list_seg_future(n,f_h,fh_ndarr):    #if n=2; a[2]=4; tlist=5:29; firstpart=5:23;lastpart=0:5
    t_part_list=[]
    i=1
    tlist=fh_ndarr[n]
    def creating_tlist(tlist,j,ind):
            try:
                ind=tlist.index(0+j*24)
                t_part_list.append([i-(0+j*24) for i in tlist[:ind]])
                tlist=tlist[ind:]
                
                j=j+1
                return creating_tlist(tlist,j,ind)
            except ValueError:
                t_part_list.append([i-(0+j*24) for i in tlist[:24-ind]])
                return t_part_list
    t_part_list=creating_tlist(tlist,i,0)
    return t_part_list

# maybe has the same functionality as above

In [51]:
def dist_predictors(x,y, monitor_points):
    """
    Returns distance and angle for an arbitrary point in the domain
    w.r.t. monitor network. These are w.r.t. each monitor or centroid of all monitors
    
    Args:
        x: x_cooridate of arbitrary point
        y: y_coordinate of arbitrary point
        training_params: Training parameters containing locations of monitors
        centroid: if True, returns distance and angle from the centrod of the monitors
    
    Returns:
        Angles and distances 
    
    """
    
    centroid=True
    x_centroid = np.average([point[0] for point in monitor_points])
    y_centroid = np.average([point[1] for point in monitor_points])
   
    def unit_vector(vector):
        """ Returns the unit vector of the vector."""
        return vector / np.linalg.norm(vector)

    def angle_between(v1, v2):
        """ Returns the angle in radians between vectors 'v1' and 'v2'::
                >>> angle_between((1, 0, 0), (0, 1, 0))
                1.5707963267948966
                >>> angle_between((1, 0, 0), (1, 0, 0))
                0.0
                >>> angle_between((1, 0, 0), (-1, 0, 0))
                3.141592653589793
        """
        v1_u = unit_vector(v1)
        v2_u = unit_vector(v2)
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

    if centroid:   
        distance = math.sqrt((x_centroid - x)**2 + (y_centroid -y)**2)
        v1 = (x_centroid, y_centroid)
        v2 = (x,y)
        angle = angle_between(v1,v2)
        return np.array([distance, angle])
    else:
        
        distances =[math.sqrt((x - point[0])**2 + (y - point[1])**2) for point in monitor_points]
        angles = [angle_between((x,y), (point[0], point[1])) for point in monitor_points]
        da = distances + angles
        return np.array(da) 

In [52]:
#@jit(nopython=True)
def emis_predictors(x,y,i_f_l,tlist):
    d=5
    xs = np.arange(-d,d+1) + x
    ys = np.arange(-d,d+1) + y

    num_cells = len(xs)*len(ys)
    xs_indexed=get_index(all_grids_x,xs)
    ys_indexed=get_index(all_grids_y,ys)
    
    def get_emis_slice(emis_dataset,species,tlist):
            grid = np.ix_(tlist,xs_indexed,ys_indexed)
            e1 = emis_dataset[species][:]
            e2 = e1[grid]
            return e2.reshape(len(tlist), num_cells)
    for i in range(len(i_f_l)):
      emis_dataset=train_param(i_f_l[i])['emis_dataset']
      emis_species=train_param(i_f_l[i])['emis_species']
      if(i==0):
        emis_seq = [ get_emis_slice(emis_dataset, species,tlist[i]) for species in emis_species] 
        emis_seq = np.concatenate(emis_seq,axis=1)
        emis_seq_total=emis_seq
      else:
          emis_seq = [ get_emis_slice(emis_dataset, species,tlist[i]) for species in emis_species] 
          emis_seq = np.concatenate(emis_seq,axis=1)
          emis_seq_total=np.vstack((emis_seq_total,emis_seq))
    return emis_seq_total

In [53]:
def history_monitors_predictors(t,i_f_l, monitor_points):

    monitors_x, monitors_y = zip(*monitor_points) 

    monitors_x_edit=get_index(all_grids_x,monitors_x)
    monitors_y_edit=get_index(all_grids_y,monitors_y)

    for i in range(len(i_f_l)):
        aq_input_data=train_param(i_f_l[i])['aq_input_data']  
        if(i==0):
          concs = aq_input_data['O3'][t[i]][:,monitors_x_edit,monitors_y_edit]
          concs_total=concs
        else:
          concs = aq_input_data['O3'][t[i]][:,monitors_x_edit,monitors_y_edit]
          concs_total=np.vstack((concs_total,concs))  
    return concs_total

In [54]:
def met_predictors(t,i_f_l,monitor_points):
   
    monitors_x, monitors_y = zip(*monitor_points)     
    monitors_x_edit=get_index(all_grids_x,monitors_x)    #function to get index
    monitors_y_edit=get_index(all_grids_y,monitors_y)    #function to get index
    
    def get_met(met_dataset,met_params,t):
      met_seq = []
      for param in met_params:
          m1 = met_dataset[param][:]
          m2 = m1[t][:,monitors_x_edit,monitors_y_edit]
          met_seq.append(m2)
      return met_seq
    for i in range(len(i_f_l)):
        met_dataset=train_param(i_f_l[i])['met_dataset']
        met_params=train_param(i_f_l[i])['met_params']
        if(i==0):
            met_seq = get_met(met_dataset,met_params,t[i])
            met_seq=np.concatenate(met_seq, axis=1)
            met_seq_total=met_seq
        else:
            met_seq = get_met(met_dataset,met_params,t[i])
            met_seq=np.concatenate(met_seq, axis=1)
            met_seq_total=np.vstack((met_seq_total,met_seq))
    return met_seq_total

In [55]:
def get_predictors_xyt(p,tlist,i_f_l,debug=False):
    """
    For each point and timestamp(s) generate a row of predictors
    """
    
    x=p[0]
    y=p[1]
    dist = dist_predictors(x,y,monitor_points) #time invariant
    dist_tile = np.tile(dist,(sum([len(i) for i in tlist]),1))

    
    emis = emis_predictors(x,y,i_f_l,tlist)
    met  = met_predictors(tlist,i_f_l,monitor_points)
    
    hist = history_monitors_predictors(tlist,i_f_l,monitor_points)
   
    try:
        preds = np.concatenate([dist_tile, emis, hist,met], axis=1)
        return preds
    except:
        return [dist_tile,emis,hist,met]

In [56]:
def get_aq_input(x,y,t,i_f_l):
    x_indexed=get_index(all_grids_x,x)
    y_indexed=get_index(all_grids_y,y)
    
    for i in range(len(i_f_l)):
        #print(len(i_f_l))
        #print(len(t))
        
        aq_input_data=train_param(i_f_l[i])['aq_input_data']  
        if(i==0):
          aq_input = aq_input_data['O3'][t[i]][:,x_indexed,y_indexed]
          aq_input_total=aq_input
        else:
          aq_input = aq_input_data['O3'][t[i]][:,x_indexed,y_indexed]
          aq_input_total=np.concatenate((aq_input_total,aq_input))  
    return aq_input_total

In [57]:
def get_future_data(o_f_l,tlist,point_x,point_y):
    for i in range(len(o_f_l)):
      aq_output_data=train_param(o_f_l[i])['o3_dataset']
      if(i==0):
        aq_output=aq_output_data['O3'][tlist[i],point_x,point_y]
        aq_output_total=aq_output
        #print(len(aq_output))
      else:
        aq_output=aq_output_data['O3'][tlist[i],point_x,point_y]
        print(aq_output_total)
        print(aq_output)
        aq_output_total=np.concatenate((aq_output_total,aq_output))
    
    return aq_output_total    

In [58]:
def train_param(file_num):
    infile=np.load(short_dir+str(file_num)+'.npz')
    

        #print(i)
    
    #outfile=np.load(short_dir+str(out_file_list[0])+'.npz')
    #outfile=outfile['O3']
    #f_output=tempfile.TemporaryFile()
    #for i in range(1,len(out_file_list)):
    #    file_1=np.load(short_dir+str(out_file_list[i])+'.npz')
    #    np.savez(f_output,O3=create_npz(file_1,outfile,'O3'))
    #    _ = f_output.seek(0) # Only needed here to simulate closing & reopening file
    #    outfile=np.load(f_output)
    #f_output = tempfile.NamedTemporaryFile(delete=True)
    #f_output.close()
    training_params = {
                   'd' : 5, #local_emissions_size
                   'emis_species': ['NO', 'NO2'], #emission_species
                   'met_params':['PBL', 'Q2', 'TEMP2', 'WSPD10', 'WDIR10'], #met parameters
                   #'met_params': ['PBL', 'WDIR10'],
                   'emis_dataset': infile,
                   'met_dataset':infile,
                   'aq_input_data':infile,
                   'o3_dataset': infile,
                   'monitor_points':monitor_points
                 }
    return training_params

In [59]:
# In[11]:  #This function is to
def limiting_points(cell_list,grid_sample_limit):
    return cell_list[np.logical_or(np.logical_and((cell_list<grid_sample_limit[0][1]),(cell_list>grid_sample_limit[0][0])),
                                   np.logical_and((cell_list<grid_sample_limit[1][1]),(cell_list>grid_sample_limit[1][0])),
                                   np.logical_and((cell_list<grid_sample_limit[2][1]),(cell_list>grid_sample_limit[2][0])))]

In [60]:
def sampling_function(all_points,x_no,y_no):
    x_cells=[]
    y_cells=[]
    
    x_cells=random.sample(list(all_grids_x),k=x_no)
    y_cells=random.sample(list(all_grids_y),k=y_no)  
    x_limited=limiting_points(np.array(x_cells),grid_sample_limit_x)
    y_limited=limiting_points(np.array(y_cells),grid_sample_limit_y)

    x_y=list(set(product(x_limited,y_limited)))
    return x_y

In [61]:
def tqdm_function(x_y_t_f,files,a,look_back,f_h):
      predictors={}
      aq_data={}
      aq_input={}
      def round_func(val):
        if(val%1>0):
          return int(val)+1
        else:
          return int(val)
      l_edge=files[round_func(look_back/24) -1]
      r_edge=files[-1*int(f_h/24)]
      lb_ndarr,fh_ndarr=hours_list(a,look_back,f_h)
      for i in x_y_t_f:
          if((i[0]>l_edge) and (i[0]<r_edge)):
              #inp_file_list = range(i[0]-round_func(look_back/24),i[0]+1)
              #out_file_list=range(i[0],i[0]+int(f_h/24)+1)
              #o3_dataset=train_param(out_file_list)['o3_dataset']


              point=i[1][0]#[0]                 ## x y coordinates of the current point


              #hour_next = a[i[1][1]]+1  #hour to be forecasted.
              point_x=get_index(all_grids_x,int(point[0]))
              point_y=get_index(all_grids_y,int(point[1]))
              #lb_ndarr,fh_ndarr=hours_list(look_back,f_h)
              #print(list(inp_file_list))
              #print(list(out_file_list))
              #o3_dataset=o3_dataset['O3']
              hours_input=list_seg(i[1][1],look_back,lb_ndarr)
              hours_output=list_seg_future(i[1][1],f_h,fh_ndarr)
              #print(type(o3_dataset[fh_ndarr[i[1][1]],point_x,point_y]))
              inp_file_list=range(i[0]-len(hours_input),i[0])#+1)
              out_file_list=range(i[0],i[0]+len(hours_output))
              aq_data[i]=get_future_data(out_file_list,hours_output,point_x,point_y)
              print(len(aq_data[i]))
              #print(aq_data[i])
              aq_input[i]= get_aq_input(point[0],point[1],hours_input,inp_file_list)
              print(i[0])
              #print(type(get_predictors_xyt(point,lb_ndarr[i[1][1]],training_params,debug=False)))
              predictors[i]=get_predictors_xyt(point,hours_input,inp_file_list,debug=False)
              #print(len(predictors[i][1]))
              #print(predictors[i][1])
              #print(aq_data[i])
      return predictors, aq_data, aq_input

In [62]:
def feat_ret(nfiles,a,n1,n2,look_back,f_h):
    aq_data={}

    predictors={}
    aq_input={}
    x_y=[]
    x_y=sampling_function(all_points,n1,n2)
    x_y.append(tuple([58,121]))
    x_y_t=product(x_y,b)
    x_y_t=list(x_y_t)
    
    for f_no in range(len(nfiles)):
        x_y_t_f=product(nfiles[f_no],x_y_t) 

        x_y_t_f=list(x_y_t_f)
        print(len(x_y_t_f))
        print(f_no)
        predictors_temp,aq_data_temp,aq_input_temp=tqdm_function(x_y_t_f,nfiles[f_no],a,look_back,f_h)
        predictors.update(predictors_temp)
        aq_data.update(aq_data_temp)
        aq_input.update(aq_input_temp)

    return predictors,aq_data,aq_input,x_y

In [63]:
##Commented after data extraction

look_back=30
f_h=24
predictors,aq_data,aq_input,x_y=feat_ret(nfiles_train,a,25,15,look_back,f_h)
print ('\n Done getting predictors')

9345
0


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/woshi/Desktop/CE_675_Project/Data/short_data/7.npz'

In [ ]:
# ### Create Random Split for Training and Testing
save_to_file = True
read_from_file = True
analyze = True
create_plots = True

In [ ]:
def get_splits(predictors,train_fraction=0.95):
    num_predictors = len(predictors)
    pxy = list(predictors.keys())
    #print(len(aq_data))
    num_train = int(train_fraction*num_predictors)
    train_indices = np.random.choice(np.arange(num_predictors),num_train, replace=False)
    pxy_train = [pxy[i] for i in train_indices]
    predictors_train = [predictors[i] for i in pxy_train]
    aq_train=[]
    for i in pxy_train:
        aq_train.append(aq_data[i])
    #aq_train = [aq_data[i] for i in pxy_train]
    print ('number of training points', num_train, '&', len(predictors_train))

    #Random sample for Testing
    predictors_test = [predictors[i] for i in pxy if i not in pxy_train]
    aq_test = [aq_data[i] for i in pxy if i not in pxy_train]
    pxy_test = [i for i in pxy if i not in pxy_train]
    print('number of testing points', len(predictors_test), '&', len(aq_test))
    return predictors_train, aq_train, predictors_test, aq_test

In [ ]:
##Commented after data extraction


if not read_from_file:
   print('Getting train and test splits')
   predictors_train, aq_train, predictors_test, aq_test = get_splits(predictors)
### 6. Data Standerdization

# In[15]:


type(predictors_train)


# In[16]:

# =============================================================================
# 
if not read_from_file:

    print('scaling inputs and outputs')
    from sklearn.preprocessing import MinMaxScaler

    data = np.concatenate(list(predictors.values()))
    print(len(data[1]))
    pred_scaler = MinMaxScaler()
    pred_scaler.fit(data)

    print(len(aq_data))
    for i in aq_data:
      print(len(aq_data[i]))
    aq_data_1 = np.stack(aq_data.values()).reshape(-1,24)
    aq_scaler = MinMaxScaler()
    aq_scaler.fit(aq_data_1)

    predictors_train_scaled = [pred_scaler.transform(mx) for mx in predictors_train]
    predictors_train_rnn = np.stack(predictors_train_scaled)
    aq_train_rnn = aq_scaler.transform(np.stack(aq_train).reshape(-1,24))

    predictors_test_scaled = [pred_scaler.transform(mx) for mx in predictors_test]
    predictors_test_rnn = np.stack(predictors_test_scaled)
    aq_test_rnn = aq_scaler.transform(np.stack(aq_test).reshape(-1,24))

In [ ]:
# =============================================================================
### Save for future use.
predictors_file = 'C:/Users/woshi/Desktop/CE_675_Project/Data/predictors/'

In [ ]:
# =============================================================================
if save_to_file == True:

    
    data_to_save = [predictors_train_rnn, aq_train, predictors_test_rnn,aq_test] 

    np.savez(predictors_file, predictors_train_rnn = predictors_train_rnn, #X_train, shape=(samples,tsteps,features)
                              aq_train_rnn = aq_train_rnn, #y_train
                              predictors_test_rnn = predictors_test_rnn, #X_test
                              aq_test_rnn = aq_test_rnn) #y_test
# =============================================================================

In [ ]:
predictors_arrays=np.load(predictors_file)
# In[17]:
predictors_train_rnn = predictors_arrays['predictors_train_rnn']
aq_train_rnn=predictors_arrays['aq_train_rnn']
predictors_test_rnn = predictors_arrays['predictors_test_rnn']
aq_test_rnn=predictors_arrays['aq_test_rnn']
#for i in range(len(predictors_train_scaled)):
#   print(len(predictors_train_scaled[i]))

In [ ]:
# Modeling

# In[18]:

predictors_future,aq_data_future,aq_input,x_y=feat_ret(nfiles_test,a,15,15,look_back,f_h)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.utils import np_utils
import copy
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import model_from_json

In [ ]:
s0,s1,s2=predictors_train_rnn.shape
print ('shape of x_train data', s0,s1,s2)
print ('shape of y_train data', aq_train_rnn.shape)
output_dir='/mnt/raid2/System/home/mmohan3/weather_data/output_data/'
pred_scaler=load(output_dir+'pred_scaler.joblib')
aq_scaler=load(output_dir+'aq_scaler.joblib')

In [ ]:
# Design Network
def network_build(neuron,batch_size,dropout_factor,optimizer):
    def fit_lstm(neuron,batch_size,dropout_factor,optimizer):
         m = Sequential()
         m.add(LSTM(neuron,input_shape=predictors_train_rnn.shape[1:], return_sequences=True, activation='tanh'))
         m.add(LSTM(neuron, activation='tanh'))#
         m.add(Dropout(dropout_factor))#
         m.add(Dense(128, activation='tanh'))
         m.add(Dense(f_h))
         m.compile(loss='mean_squared_error',optimizer=optimizer)
         return m


# In[21]:

    model=fit_lstm(neuron,batch_size,dropout_factor,optimizer)
#  predictors_train_rnn.shape[1:]
    
    

# In[22]:
          
    def summary_history(m):
        print(m.summary())
        print("Inputs: {}".format(m.input_shape))
        print( "Outputs: {}".format(m.output_shape))
        history = m.fit(predictors_train_rnn, aq_train_rnn, epochs=70, 
                        validation_data=(predictors_test_rnn, aq_test_rnn),batch_size=batch_size, verbose=2, shuffle=True)           
        return None
    summary_history(model)
    nns_dir='/mnt/raid2/System/home/mmohan3/weather_data/neural_networks/'
    model_json = model.to_json()
    with open(nns_dir+"model"+str(neuron)+str(batch_size)+str(dropout_factor)+optimizer+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(nns_dir+"model_"+str(neuron)+str(batch_size)+str(dropout_factor)+optimizer+".h5")
    print("Saved model to disk")

# In[29]:      Performance evaluation part
# ### Testing Using Future Days Data
# In[31]:
## Scale, Predict, and Descale:
      #  predictors_future,aq_data_future,aq_input,x_y=feat_ret(nfiles_test,a,15,15,look_back,f_h)
    print ('\n Done getting predictors') 
    X = list(predictors_future.values())
    print(len(X))
    X = list(aq_data_future.values())
    print(len(X))
    
    
    ## Scale, Predict, and Descale:
    
    def get_model_predictions(model, X_as_dict, X_scaler, y_scaler):
        X = list(X_as_dict.values())
        X_temp = [X_scaler.transform(mx) for mx in X]
        X_scaled = np.stack(X_temp)
        y_scaled = model.predict(X_scaled)
        return y_scaler.inverse_transform(y_scaled)
    
    y_model_predicted = get_model_predictions(model, predictors_future, pred_scaler, aq_scaler)
    y = np.stack(aq_data_future.values()).reshape(-1,f_h)
    
    y_labels=list(aq_data_future.keys())
    
    y_pred_dict={}
    print(y_labels)
    for i in range(len(y_labels)):
        y_pred_dict[y_labels[i]]=y_model_predicted[i]

    output_dir='/mnt/raid2/System/home/mmohan3/weather_data/output_data/'
    day_c=[]
    d=198
    for i in range(no_of_months):
        day_c.append(d)
        d=d+30
    b=[2,7]
    loc_c=tuple([58,121])
    loc_b=[]
    for i in b:
        loc_b.append(tuple([loc_c,i]))
    
    b_loc_day=list(product(day_c,loc_b))
    months=['July','August','September','October']
    
    df=pd.DataFrame()
    for i in range(no_of_months):
        fig = plt.figure(figsize=(10,10))
        plt.rcParams['axes.linewidth'] = 5
        ax = fig.add_subplot(1,1,1)
        colors=['b','g']
        df=pd.DataFrame()
        
        for j in range(2):
            y_pred_trend =y_pred_dict[b_loc_day[i * 2 + j]]
            y_obs_trend =  aq_data_future[b_loc_day[i*2+j]]
            df['aq_input'+str(b_loc_day[i*2+j])]=aq_input[b_loc_day[i * 2 + j]]
            df['y_pred_trend'+str(b_loc_day[i*2+j])]=pd.Series(y_pred_dict[b_loc_day[i * 2 + j]])
            df['y_obs_trend'+str(b_loc_day[i*2+j])]=pd.Series(aq_data_future[b_loc_day[i * 2 + j]])
        fig.show()
        ax.set_ylim([0.00,0.09])
        ax.set_ylabel('Hr', fontdict={'fontsize':10}, labelpad=20)
        ax.set_xlabel('Concentration (ppm)', fontdict={'fontsize':10}, labelpad=20)
        ax.set_title('Fig 5b. Model Predictions vs \n Observed on Test Dataset for a 24 Hr Forecast - '+months[i], fontdict={'fontsize':15},y=1.08)
        fig.savefig(output_dir+str(neuron)+str(batch_size)+str(dropout_factor)+'6b-'+str(i)+'.pdf', format='pdf', dpi=1200)
    df.to_excel(output_dir+str(neuron)+str(batch_size)+str(dropout_factor)+'output_file.xlsx')
    fig=plt.figure(figsize=(8,8))
    plt.rcParams['axes.linewidth'] = 5
    ax = fig.add_subplot(1,1,1)
    rmse_list=[]
    for i in range(no_of_months):
        select_points=random.sample(x_y,k=5)
        hours=[2,3,6,9]
        point_hour=list(product(select_points,hours))
        rmse=[]
        for j in point_hour:
            pred_data=y_pred_dict[tuple([day_c[i],j])]
            aq_data=aq_data_future[tuple([day_c[i],j])]
            mse=mean_squared_error(pred_data,aq_data)
            rmse.append(np.sqrt(mse))
        rmse_list.append(rmse)
    plt.boxplot(rmse_list)
    plt.title('Average RMSE for the first day of JASO months sampled over 5 different coordinates', fontdict={'fontsize':10},y=1.08)
    fig.savefig(output_dir+str(neuron)+str(batch_size)+str(dropout_factor)+'7b.pdf', format='pdf', dpi=1200)
    return None

In [ ]:
neurons=[16,32,64,128]
batch_sizes=[15,30,45,60,75,90,105,120]
dropout_factors=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
optimizers=['adam','RMSProp']

for i in batch_sizes:
    neuron=64
    batch_size=60
    dropout_factor=0.5
    optimizer='RMSProp'
    network_build(i,batch_size,dropout_factor,optimizer)